In [1]:
import os
import numpy as np
from utils import NearestNeighbour
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
fns = [os.path.join('data', fn) for fn in os.listdir('data')]
fns_training = list(filter(lambda x: 'data_batch' in x, fns))
data = [unpickle(fn) for fn in fns_training]
fns_test = list(filter(lambda x: 'test_batch' in x, fns))
data_test = [unpickle(fn) for fn in fns_test]

In [4]:
x = torch.tensor([1,2,3], dtype=torch.float32)

In [ ]:

# lets split the data into training, validation and test
X = np.concatenate([data[i][b'data'] for i in range(len(data))], axis=0)
Y = np.concatenate([data[i][b'labels'] for i in range(len(data))], axis=0)
Xtr = torch.tensor(X[0:45000, :], dtype=torch.float32)
Ytr = torch.tensor(Y[0:45000])
Xdev = torch.tensor(X[45000:50000, :])
Ydev = torch.tensor(Y[45000:50000])
Xtest = torch.tensor(np.concatenate([data_test[i][b'data'] for i in range(len(data_test))], axis=0))
Ytest = torch.tensor(np.concatenate([data_test[i][b'labels'] for i in range(len(data_test))], axis=0))

: 

: 

In [7]:
class NeuralNetwork(nn.Module):
        def __init__(self):
                super(NeuralNetwork, self).__init__()
                self.mlp = nn.Sequential(
                        nn.Linear(3072, 100), nn.ReLU(),
                        nn.Linear(100, 100), nn.ReLU(),
                        nn.Linear(100, 10)
                )

        def forward(self, x):
                logits = self.mlp(x)
                return logits

In [10]:
model1 = nn.Sequential(
                        nn.Linear(3072, 100), nn.Tanh(),
                        nn.Linear(100, 100), nn.Tanh(),
                        nn.Linear(100, 10)
                )


In [8]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (mlp): Sequential(
    (0): Linear(in_features=3072, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [11]:
model1(torch.randn(1, 3072))

tensor([[ 0.0744, -0.1204,  0.4186,  0.0643, -0.0642,  0.0933, -0.1860, -0.0784,
         -0.2194,  0.0496]], grad_fn=<AddmmBackward0>)

In [14]:
torch.tensor(np.zeros((10,10    )), dtype=torch.float32)  

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])